# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core import Metrics

%matplotlib inline
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

In [2]:
simulation_name = "example_dinwoodie"

sim = Simulation(simulation_name, "DINWOODIE", "base.yaml")
sim.run()

# For convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 95.0%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

S00T1  S00T2  S00T3  S00T4  S00T5  S00T6  S00T7  S00T8  S00T9  S00T10  \
0   0.94   0.96   0.99   0.93   0.97   0.90   0.98   0.98   0.98    0.97   

   S00T11  S00T12  S00T13  S00T14  S00T15  S00T16  S00T17  S00T18  S00T19  \
0    0.96    0.98    0.86    0.98    0.98    0.93    0.98    0.94    0.86   

   S00T20  S00T21  S00T22  S00T23  S00T24  S00T25  S00T26  S00T27  S00T28  \
0    0.98    0.98    0.89    0.95    0.97    0.88    0.98    0.98    0.98   

   S00T29  S00T30  S00T31  S00T32  S00T33  S00T34  S00T35  S00T36  S00T37  \
0    0.92    0.94    0.94    0.98    0.98    0.98    0.98    0.85    0.98   

   S00T38  S00T39  S00T40  S00T41  S00T42  S00T43  S00T44  S00T45  S00T46  \
0    0.91    0.98    0.98    0.98    0.98    0.97    0.80    0.98    0.97   

   S00T47  S00T48  S00T49  S00T50  S00T51  S00T52  S00T53  S00T54  S00T55  \
0    0.98    0.98    0.96    0.98    0.98    0.89    0.97    0.92    0.97   

   S00T56  S00T57  S00T58  S00T59  S00T60  S00T61  S00T62  S00T63  S00T64  \
0    0.92    0.98    0.91    0.98    0.98    0.95    0.95    0.98    0.94   

   S00T65  S00T66  S00T67  S00T68  S00T69  S00T70  S00T71  S00T72  S00T73  \
0    0.88    0.95    0.93    0.97    0.98    0.98    0.89    0.97    0.94   

   S00T74  S00T75  S00T76  S00T77  S00T78  S00T79  S00T80  
0    0.96    0.92    0.97    0.93    0.89    0.98    0.89

In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,year,windfarm
0,2003,0.98
1,2004,0.95
2,2005,0.95
3,2006,0.97
4,2007,0.94
5,2008,0.95
6,2009,0.97
7,2010,0.97
8,2011,0.91
9,2012,0.90


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,month,windfarm
0,1,0.95
1,2,0.95
2,3,0.94
3,4,0.93
4,5,0.95
5,6,0.95
6,7,0.95
7,8,0.95
8,9,0.96
9,10,0.97


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

,year,month,windfarm
0,2003,1,0.99
1,2003,2,0.99
2,2003,3,0.99
3,2003,4,0.97
4,2003,5,0.98
5,2003,6,0.99
6,2003,7,0.98
7,2003,8,0.98
8,2003,9,0.97
9,2003,10,0.96


## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.45%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

S00T1  S00T2  S00T3  S00T4  S00T5  S00T6  S00T7  S00T8  S00T9  S00T10  \
0   0.45   0.46   0.47   0.45   0.46   0.43   0.47   0.47   0.46    0.46   

   S00T11  S00T12  S00T13  S00T14  S00T15  S00T16  S00T17  S00T18  S00T19  \
0    0.45    0.46    0.41    0.46    0.47    0.44    0.47    0.44    0.41   

   S00T20  S00T21  S00T22  S00T23  S00T24  S00T25  S00T26  S00T27  S00T28  \
0    0.47    0.46    0.42    0.45    0.46    0.42    0.46    0.46    0.46   

   S00T29  S00T30  S00T31  S00T32  S00T33  S00T34  S00T35  S00T36  S00T37  \
0    0.44    0.44    0.45    0.47    0.47    0.47    0.47    0.40    0.47   

   S00T38  S00T39  S00T40  S00T41  S00T42  S00T43  S00T44  S00T45  S00T46  \
0    0.42    0.46    0.47    0.47    0.46    0.46    0.38    0.47    0.46   

   S00T47  S00T48  S00T49  S00T50  S00T51  S00T52  S00T53  S00T54  S00T55  \
0    0.47    0.47    0.46    0.47    0.47    0.42    0.46    0.44    0.47   

   S00T56  S00T57  S00T58  S00T59  S00T60  S00T61  S00T62  S00T63  S00T64  \
0    0.43    0.47    0.43    0.47    0.46    0.46    0.46    0.47    0.45   

   S00T65  S00T66  S00T67  S00T68  S00T69  S00T70  S00T71  S00T72  S00T73  \
0    0.42    0.46    0.45    0.46    0.46    0.47    0.43    0.46    0.44   

   S00T74  S00T75  S00T76  S00T77  S00T78  S00T79  S00T80  
0    0.46    0.44    0.46    0.45    0.42    0.47    0.43

In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.47
2006,0.46
2007,0.48
2008,0.49
2009,0.46
2010,0.41
2011,0.46


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.49
3,0.50
4,0.40
5,0.38
6,0.32
7,0.32
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.38
     4          0.53
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.53
     12         0.54
2004 1          0.53
     2          0.49
     3          0.49
     4          0.38
     5          0.29
     6          0.37
     7          0.27
     8          0.41
     9          0.57
     10         0.55
     11         0.47
     12         0.49

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,year,Completion Rate
0,2003,0.99
1,2004,1.00
2,2005,1.00
3,2006,1.00
4,2007,0.99
5,2008,1.01
6,2009,0.99
7,2010,1.01
8,2011,0.98
9,2012,1.00


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,year,Completion Rate
0,1,0.98
1,2,0.99
2,3,1.01
3,4,0.99
4,5,1.03
5,6,1.01
6,7,1.00
7,8,1.01
8,9,0.98
9,10,1.03


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

,year,month,Completion Rate
0,2003,1,0.91
1,2003,2,1.09
2,2003,3,0.94
3,2003,4,1.07
4,2003,5,0.99
5,2003,6,1.00
6,2003,7,0.99
7,2003,8,1.03
8,2003,9,0.94
9,2003,10,0.93


## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $488,967.84/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","95,512,500.00"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,year,equipment_cost
0,2003,"11,227,531.25"
1,2004,"10,437,500.00"
2,2005,"11,182,250.00"
3,2006,"10,619,750.00"
4,2007,"13,226,000.00"
5,2008,"11,318,750.00"
6,2009,"10,882,250.00"
7,2010,"11,332,250.00"
8,2011,"11,807,250.00"
9,2012,"14,450,000.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1,"543,010.42","543,010.42","543,010.42",0.00,0.00
1,2,"495,250.00","495,250.00","495,250.00",0.00,0.00
2,3,"542,500.00","542,500.00","542,500.00",0.00,0.00
3,4,"525,000.00","525,000.00","525,000.00",0.00,0.00
4,5,"542,500.00","542,500.00","542,500.00",0.00,"33,018,750.00"
5,6,"525,000.00","525,000.00","525,000.00","2,670,291.67",0.00
6,7,"542,500.00","542,500.00","542,500.00",0.00,"30,762,500.00"
7,8,"542,500.00","542,500.00","542,500.00",0.00,0.00
8,9,"525,000.00","525,000.00","525,000.00",0.00,"31,731,250.00"
9,10,"542,500.00","542,500.00","542,500.00",0.00,0.00


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

,year,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,2003,1,"54,760.42","54,760.42","54,760.42",0.00,0.00
1,2003,2,"49,000.00","49,000.00","49,000.00",0.00,0.00
2,2003,3,"54,250.00","54,250.00","54,250.00",0.00,0.00
3,2003,4,"52,500.00","52,500.00","52,500.00",0.00,0.00
4,2003,5,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
5,2003,6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
6,2003,7,"54,250.00","54,250.00","54,250.00",0.00,"3,543,750.00"
7,2003,8,"54,250.00","54,250.00","54,250.00",0.00,0.00
8,2003,9,"52,500.00","52,500.00","52,500.00",0.00,"2,750,000.00"
9,2003,10,"54,250.00","54,250.00","54,250.00",0.00,0.00


## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.45,0.42,0.43,0.18,0.81


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.43,0.39,0.43,0.00,0.63
2004,0.57,0.60,0.63,0.00,0.65
2005,0.51,0.46,0.49,0.14,1.00
2006,0.50,0.45,0.46,0.32,0.36
2007,0.53,0.47,0.41,0.21,1.00
2008,0.50,0.43,0.52,0.07,1.00
2009,0.36,0.35,0.39,0.39,0.69
2010,0.43,0.33,0.34,0.36,0.50
2011,0.35,0.38,0.33,0.14,1.00


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,year,total_labor_cost
0,2003,0.00
1,2004,0.00
2,2005,0.00
3,2006,0.00
4,2007,0.00
5,2008,0.00
6,2009,0.00
7,2010,0.00
8,2011,0.00
9,2012,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,month,total_labor_cost
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
5,6,0.00
6,7,0.00
7,8,0.00
8,9,0.00
9,10,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

,year,month,total_labor_cost
0,2003,1,0.00
1,2003,2,0.00
2,2003,3,0.00
3,2003,4,0.00
4,2003,5,0.00
5,2003,6,0.00
6,2003,7,0.00
7,2003,8,0.00
8,2003,9,0.00
9,2003,10,0.00


## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,reason,total_cost
0,Maintenance,"2,069,375.00"
1,Repair,"22,743,072.92"
2,Mobilization,"15,000,000.00"
3,Weather Delay,"12,845,093.75"
4,No Requests,"27,235,093.75"
5,Not in Shift,"37,459,645.83"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,reason,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
0,Maintenance,0.00,0.00,0.00,"2,069,375.00","2,069,375.00"
1,Repair,0.00,0.00,0.00,"22,743,072.92","22,743,072.92"
2,Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
3,Weather Delay,0.00,0.00,0.00,"12,845,093.75","12,845,093.75"
4,No Requests,0.00,0.00,0.00,"27,235,093.75","27,235,093.75"
5,Not in Shift,0.00,0.00,0.00,"37,459,645.83","37,459,645.83"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

,year,reason,total_cost
0,2003,Maintenance,"179,229.17"
1,2003,Repair,"1,898,875.00"
2,2003,Mobilization,"1,500,000.00"
3,2003,Weather Delay,"910,229.17"
4,2003,No Requests,"3,988,677.08"
5,2003,Not in Shift,"2,750,520.83"
6,2004,Maintenance,"306,395.83"
7,2004,Repair,"1,899,458.33"
8,2004,Mobilization,"1,500,000.00"
9,2004,Weather Delay,"665,739.58"


In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

,month,reason,total_cost
0,1,Maintenance,"181,489.58"
1,1,Repair,"209,125.00"
2,1,Mobilization,0.00
3,1,Weather Delay,"93,697.92"
4,1,No Requests,"709,333.33"
5,1,Not in Shift,"435,385.42"
6,2,Maintenance,"133,072.92"
7,2,Repair,"209,416.67"
8,2,Mobilization,0.00
9,2,Weather Delay,"67,083.33"


In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

,year,month,reason,total_cost
0,2003,1,Maintenance,0.00
1,2003,1,Repair,"21,583.33"
2,2003,1,Mobilization,0.00
3,2003,1,Weather Delay,"7,510.42"
4,2003,1,No Requests,"111,708.33"
5,2003,1,Not in Shift,"23,479.17"
6,2003,2,Maintenance,0.00
7,2003,2,Repair,"18,447.92"
8,2003,2,Mobilization,0.00
9,2003,2,Weather Delay,"5,760.42"


## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,component,total_cost
0,cable,0.00
1,drive_train,0.00
2,electrical_system,0.00
3,electronic_control,0.00
4,gearbox,0.00
5,generator,"112,320,656.25"
6,hydraulic_system,0.00
7,mechanical_brake,0.00
8,rotor_blades,0.00
9,rotor_hub,0.00


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

,component,action,materials_cost,total_labor_cost,equipment_cost,total_cost
0,generator,delay,0,0.00,"50,303,208.33","50,303,208.33"
1,generator,maintenance,0,0.00,"2,069,375.00","2,069,375.00"
2,generator,repair,0,0.00,"22,743,072.92","22,743,072.92"


In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,year,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,cable,0.00,0.00,0.00,0.00
1,2003,drive_train,0.00,0.00,0.00,0.00
2,2003,electrical_system,0.00,0.00,0.00,0.00
3,2003,electronic_control,0.00,0.00,0.00,0.00
4,2003,gearbox,0.00,0.00,0.00,0.00
5,2003,generator,"3,006,000.00",0.00,"5,737,322.92","8,743,322.92"
6,2003,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,rotor_blades,0.00,0.00,0.00,0.00
9,2003,rotor_hub,0.00,0.00,0.00,0.00


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,1,cable,0.00,0.00,0.00,0.00
1,1,drive_train,0.00,0.00,0.00,0.00
2,1,electrical_system,0.00,0.00,0.00,0.00
3,1,electronic_control,0.00,0.00,0.00,0.00
4,1,gearbox,0.00,0.00,0.00,0.00
5,1,generator,"1,299,500.00",0.00,"918,166.67","2,217,666.67"
6,1,hydraulic_system,0.00,0.00,0.00,0.00
7,1,mechanical_brake,0.00,0.00,0.00,0.00
8,1,rotor_blades,0.00,0.00,0.00,0.00
9,1,rotor_hub,0.00,0.00,0.00,0.00


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

,year,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,1,cable,0.00,0.00,0.00,0.00
1,2003,1,drive_train,0.00,0.00,0.00,0.00
2,2003,1,electrical_system,0.00,0.00,0.00,0.00
3,2003,1,electronic_control,0.00,0.00,0.00,0.00
4,2003,1,gearbox,0.00,0.00,0.00,0.00
5,2003,1,generator,"58,000.00",0.00,"51,041.67","109,041.67"
6,2003,1,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,1,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,1,rotor_blades,0.00,0.00,0.00,0.00
9,2003,1,rotor_hub,0.00,0.00,0.00,0.00


## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,year,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
1,2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2,2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
3,2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
4,2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
5,2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
6,2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
7,2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
8,2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
9,2012,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"80,535.00","62,204.00","58,474.00",473.00
major repair,"73,605.65","1,241.00","1,136.00",25.00
major replacement,"202,325.32","10,375.00","201,797.32",67.00
manual reset,"148,701.48","33,430.00","12,387.00","5,553.00"
medium repair,"12,724.02","10,120.00","8,110.00",227.00
minor repair,"56,523.22","28,381.00","16,462.00","2,160.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,519,801,937.50"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,519,801,937.50","118,533,048.00","120,824,459.00","124,006,660.00","117,291,895.50","121,739,203.00","113,135,439.50","122,559,932.50","122,754,614.00","121,690,215.50","121,822,603.50","118,922,678.00","122,223,999.50","107,911,175.00","122,301,972.50","122,531,394.00","116,636,132.00","122,369,204.00","116,628,630.00","108,089,687.50","122,358,065.00","122,185,786.00","110,805,401.50","119,643,255.00","121,601,911.00","110,521,284.50","122,151,888.50","122,069,543.00","122,201,256.00","115,156,422.50","116,512,410.00","118,565,952.00","122,840,414.50","122,641,833.50","122,513,806.50","123,089,564.00","106,431,499.00","122,746,613.00","110,482,081.00","122,122,153.00","123,151,121.50","123,387,410.50","122,249,977.50","121,348,778.50","100,188,101.50","122,916,683.00","121,898,226.50","122,822,344.00","123,388,863.50","121,044,175.50","122,389,202.00","122,540,103.50","109,661,113.50","121,206,047.00","115,595,330.50","122,527,526.00","114,385,311.00","122,928,141.50","114,379,777.00","122,384,269.50","122,162,340.00","120,049,048.50","120,097,262.50","123,007,385.50","118,621,610.50","109,702,165.00","119,794,688.00","117,892,886.50","121,579,604.50","122,200,670.00","122,971,113.50","112,465,175.00","121,790,085.00","116,059,855.00","121,001,609.00","115,684,651.50","121,812,361.50","117,999,553.50","110,031,318.00","122,835,589.00","113,036,379.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"862,850,493.00"
2004,"932,354,785.50"
2005,"980,800,011.00"
2006,"957,043,425.50"
2007,"1,010,221,386.50"
2008,"1,042,266,349.00"
2009,"966,387,131.50"
2010,"865,062,267.00"
2011,"966,795,750.50"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,063,303,680.50"
2,"806,545,386.00"
3,"879,358,210.50"
4,"688,407,506.50"
5,"674,474,965.50"
6,"544,759,701.50"
7,"580,060,970.50"
8,"636,998,547.00"
9,"791,728,286.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,726,369.50
     2      59,189,884.00
     3      67,490,361.00
     4      91,689,020.00
     5      62,100,676.00
     6      53,868,174.00
     7      53,232,049.50
     8      52,928,428.00
     9      46,691,262.00
     10     75,653,501.50
     11     92,270,608.50
     12     96,010,159.00
2004 1      94,203,616.50
     2      82,087,998.50
     3      88,040,729.00
     4      65,048,555.50
     5      51,535,103.00
     6      64,150,850.00
     7      47,963,430.00
     8      73,109,760.50
     9      98,989,041.00
     10     98,126,317.50
     11     81,787,036.50
     12     87,312,347.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> A PySAM file needs to be defined in your library's windfarm directory. This part of the analysis is highly dependent on the inputs provided, and there is not a validated input data sheet used, this is merely a functionality demonstration.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
npv = metrics.pysam_npv()
print(f"NPV: ${npv:,.0f}")

NPV: $32,692,500


### Real Levelized Cost of Energy (LCOE)

In [51]:
lcoe = metrics.pysam_lcoe_real()
print(f"Real LCOE: ${lcoe:,.2f}/kW")

Real LCOE: $-0.00/kW


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
lcoe = metrics.pysam_lcoe_nominal()
print(f"Nominal LCOE: ${lcoe:,.2f}/kW")

Nominal LCOE: $-0.01/kW


### After-tax Internal Return Rate (IRR)

In [53]:
npv = metrics.pysam_irr()
print(f"IRR: {npv:,.1f}%")

IRR: 14.4%


### One Data Frame to Rule Them All

In [54]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"32,692,499.68"
Nominal LCOE ($/kW),-0.01
Real LCOE ($/kW),-0.00
After Tax IRR (%),14.43


In [55]:
sim.env.cleanup_log_files(log_only=False)